In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_profiling as pp
import seaborn as sns
%matplotlib inline

# Machine learning libraries
from sklearn.linear_model import LogisticRegression, SGDRegressor, BayesianRidge
from sklearn.svm import SVR, LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import mean_squared_error
import math

In [ ]:
# load data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
# sns.distplot(train_df['SalePrice'])

In [ ]:
# skewness is a measures of distortion from the normal distribution
print("skewness: %f" %train_df['SalePrice'].skew())
# kurtosis is used to measure outilers
print("kurtosis: %f" %train_df['SalePrice'].kurt())

skewness: 1.882876
kurtosis: 6.536282


In [ ]:
# histogram of tagert and norm
# from scipy import stats
# sns.distplot(train_df['SalePrice'], fit=stats.norm);
# fig = plt.figure()
# res = stats.probplot(train_df['SalePrice'], plot=plt)

In [ ]:
classifiers = [
                # LogisticRegression(max_iter=500),
                # SGDRegressor(),
                BayesianRidge(),
                SVR(),
                LinearSVR(),
                RandomForestRegressor(),
                DecisionTreeRegressor(),
                KNeighborsRegressor()
              ]

In [ ]:
# Parameters
# Cateogorical values have similar values
coulumns_more_similar_values = ['Id', 'Heating', 'BsmtCond', 'CentralAir', 'Electrical', 'Functional',
'Street', 'LandContour', 'Utilities', 'LandSlope', 'Condition1', 'Condition2', 'RoofMatl',
'ExterCond','BsmtFinType2', 'GarageQual', 'GarageCond', 
'PavedDrive', 'SaleType', 'SaleCondition', 'Neighborhood']

# cloumns have more than 15% of missing data
missing_data_cols1 = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'LotFrontage']

missing_data_cols2 = [ 'GarageCond', 'GarageType', 'MasVnrArea', 'MasVnrType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'BsmtUnfSF', 'Utilities',
'BsmtExposure', 'BsmtFinType2', 'BsmtFinType1', 'BsmtCond' ,'BsmtQual', 'BsmtHalfBath', 'BsmtFullBath']

#Many zeros and not much corelation with target
many_zeros = ['BsmtFinSF2', 'LowQualFinSF', 'BsmtHalfBath', 'KitchenAbvGr', 
'3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'EnclosedPorch']

similar_features = ['TotRmsAbvGrd', 'GarageCars', 'TotalBsmtSF', 'Neighborhood']

drop_cols = missing_data_cols1
drop_cols.append('Id')
print(len(drop_cols))
target = 'SalePrice'
has_cv = True
split_size = 0.8 if has_cv else 1

7


In [ ]:
percentage = (train_df.isnull().sum()).sort_values(ascending = False)
# percentage.head(20)

In [ ]:
# train_df[drop_cols].
# val = 'BsmtFinSF2'
# data = pd.concat([train_df[target], train_df[val]], axis = 1)
# data.plot.scatter(x=val, y=target, ylim=(0, 800000))

In [ ]:
# val = 'CentralAir'
# data = pd.concat([train_df[target], train_df[val]], axis = 1)
# fig = sns.boxplot(x=val, y=target, data=data)
# fig.axis(ymin=0, ymax=800000)

In [ ]:
# corr_max = train_df[many_zeros].corr()
# sns.heatmap(corr_max, vmax=.8, square=True)

In [ ]:
# cols_temp = many_zeros +  missing_data_cols
# corrmat = train_df.drop(cols_temp, axis=1).corr()
# #saleprice correlation matrix
# k = 10 #number of variables for heatmap
# cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
# cm = np.corrcoef(train_df[cols].values.T)
# sns.set(font_scale=1.25)
# hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
# plt.show()

In [ ]:
train_mod_df = train_df.drop(drop_cols, axis=1)
test_mod_df = test_df.drop(drop_cols, axis=1)

combine = [train_mod_df, test_mod_df]
from scipy.stats import skew

for data in combine:
  skewed_feats = data.select_dtypes(exclude=['object']).apply(lambda x: skew(x.dropna())) #compute skewness
  skewed_feats = skewed_feats[skewed_feats > 0.75]
  skewed_feats = skewed_feats.index

  data[skewed_feats] = np.log1p(data[skewed_feats])

In [ ]:
combine = [train_mod_df, test_mod_df]

for data in combine:
  data['BsmtFinSF1'].fillna(train_mod_df['BsmtFinSF1'].dropna().mean(), inplace=True)
  data['GarageArea'].fillna(train_mod_df['GarageArea'].dropna().mean(), inplace=True)
  data['TotalBsmtSF'].fillna(train_mod_df['TotalBsmtSF'].dropna().mode()[0], inplace=True)
  data['GarageCars'].fillna(train_mod_df['GarageCars'].dropna().mode()[0], inplace=True)
  data['TotalArea'] = data['TotalBsmtSF'] + data['1stFlrSF'] + data['2ndFlrSF'] + data['GrLivArea'] +data['GarageArea']

  data['Bathrooms'] = data['FullBath'] + data['HalfBath']*0.5 

  data['Year average']= (data['YearRemodAdd']+data['YearBuilt'])/2

  cols_numeric = list(data.select_dtypes(exclude=['object']).columns)
  for col in cols_numeric:
    data[col].fillna(0, inplace=True)
  
  cols_object = list(data.select_dtypes(include=['object']).columns)
  for col in cols_object:
    freq = train_mod_df[col].dropna().mode()[0]
    data[col].fillna(freq, inplace=True)
    # data[col] = data[col].astype('category').cat.codes
    

In [ ]:
train_dum_shape = train_mod_df.shape[0]
c = pd.concat((train_mod_df, test_mod_df), sort=False).reset_index(drop=True) 
dummies = pd.get_dummies(c)
train_mod_df = dummies[:train_dum_shape]
test_mod_df = dummies[train_dum_shape:]
print(f'train shape after dummies {train_mod_df.shape}')
print(f'test shape after dummies {test_mod_df.shape}')
percentage = (test_mod_df.isnull().sum()).sort_values(ascending = False)
percentage.head(15)

train shape after dummies (1460, 273)
test shape after dummies (1459, 273)


SalePrice                1459
SaleCondition_Partial       0
Condition1_PosN             0
Neighborhood_Timber         0
Neighborhood_Veenker        0
Condition1_Artery           0
Condition1_Feedr            0
Condition1_Norm             0
Condition1_PosA             0
Condition1_RRAe             0
Neighborhood_ClearCr        0
Condition1_RRAn             0
Condition1_RRNe             0
Condition1_RRNn             0
Condition2_Artery           0
dtype: int64

In [ ]:
# fig = plt.figure(figsize=(15, 15))
# col1 = 'TotRmsAbvGrd'
# col2 = target
# val = 13
# print(len(train_mod_df[col1]))
# ax1 = plt.subplot2grid((3, 2), (0, 0))
# plt.scatter(x=train_df[col1], y = train_df[col2])
# plt.axvline(x=val, color='r', linestyle='-')
# plt.title(col1+col2)

In [ ]:
train_mod_df['GrLivArea'].sort_values(ascending=False).head(2)
train_mod_df['TotalBsmtSF'].sort_values(ascending=False).head(1)
train_mod_df['1stFlrSF'].sort_values(ascending=False).head(1)
train_mod_df['GarageArea'].sort_values(ascending=False).head(4)
train_mod_df['TotRmsAbvGrd'].sort_values(ascending=False).head(1)

635    14.0
Name: TotRmsAbvGrd, dtype: float64

In [ ]:
pos = [1298, 593, 581, 1190, 1061, 635]
train_mod_df.drop(train_mod_df.index[pos], inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
# Get only sepcific columns
def process_data (data, test=False) :
  # Data normalization and converting pandas to numpy array
  # temp = data.select_dtypes(exclude=['object'])
  temp = data
  temp = temp.drop(target, axis=1)

  # temp = pd.get_dummies(temp)
  # temp.fillna(value=0, inplace=True)
  temp = temp.to_numpy()
  # temp = temp / np.amax(temp, axis=0)

  if (test):
    return temp
  else:
    labels = data[target]
    return temp, labels.to_numpy()

In [ ]:
# Total data
train_data_input, train_data_label = process_data(train_mod_df)
test_data_input = process_data(test_mod_df, True)
print(f'Total Train Input Shape : {train_data_input.shape}')
print(f'Total Train Label Shape : {train_data_label.shape}')
print(f'Total Test Input Shape : {test_data_input.shape}')


Total Train Input Shape : (1454, 272)
Total Train Label Shape : (1454,)
Total Test Input Shape : (1459, 272)


In [ ]:
# Split data
split = int(len(train_data_input)*split_size)
# Train split
train_input = train_data_input[:split]
train_label = train_data_label[:split]
# CV split
cv_input = train_data_input[split:]
cv_label = train_data_label[split:]

print(f'Train Input Shape : {train_input.shape}')
print(f'Train Label Shape : {train_label.shape}')
print(f'CV Input Shape : {cv_input.shape}')
print(f'CV label Shape : {cv_label.shape}')


Train Input Shape : (1163, 272)
Train Label Shape : (1163,)
CV Input Shape : (291, 272)
CV label Shape : (291,)


In [ ]:
def get_scores(cv):
  train_scores = []
  test_scores = []
  for clf in classifiers:
    clf.fit(train_input, train_label)
    pred = clf.predict(train_input)
    tscore = np.round(math.sqrt(mean_squared_error(train_label, pred)), 4)
    train_scores.append(str(tscore))
    if (cv) :
      pred = clf.predict(cv_input)
      cvscore = np.round(math.sqrt(mean_squared_error(cv_label, pred)), 4)
      test_scores.append(str(cvscore))
    else:
      test_scores.append(str(tscore))
  return train_scores, test_scores

In [ ]:
# train_scores, test_scores = get_scores(has_cv)

In [ ]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(150, input_shape=[train_input.shape[1]], activation="relu"),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(100, activation="relu"),
#     tf.keras.layers.Dense(1)
# ])
# model.summary()

# optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
# model.compile(optimizer=optimizer, loss='mse')

# if has_cv:
#   history = model.fit(train_input, train_label, validation_data=(cv_input, cv_label), epochs=500, verbose=2)
# else:
#   history = model.fit(train_input, train_label, epochs=500, verbose=2)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 150)               40950     
_________________________________________________________________
dropout_2 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 100)               15100     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 101       
Total params: 56,151
Trainable params: 56,151
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
37/37 - 0s - loss: 62555.8867 - val_loss: 157.4976
Epoch 2/500
37/37 - 0s - loss: 222.5246 - val_loss: 19.6739
Epoch 3/500
37/37 - 0s - loss: 71.5441 - val_loss: 3.4738
Epoch 4/500
37/37 - 0s - loss: 33.7639 - val_loss: 9.5909
Epoch 5/5

KeyboardInterrupt: ignored

In [ ]:
names = list(map(lambda x: x.__class__.__name__, classifiers))
# names.append("Neural network")
models = pd.DataFrame({
    'Models': names,
    'Train-accuracy': train_scores,
    'Validation-accuracy': test_scores 
})
models.sort_values(by='Validation-accuracy')

,Models,Train-accuracy,Validation-accuracy
4,DecisionTreeRegressor,0.0,0.0
3,RandomForestRegressor,0.0529,0.0529
0,BayesianRidge,0.0984,0.0984
5,KNeighborsRegressor,0.2026,0.2026
1,SVR,0.2751,0.2751
2,LinearSVR,0.3021,0.3021


In [ ]:
def getSubmission (value):
  for clf in classifiers:
    if (clf.__class__.__name__ == value):
      print(clf.__class__.__name__)
      predictions = clf.predict(test_data_input)
  return predictions

In [ ]:
def getNNSubmission ():
  pred = model.predict(test_data_input)
  return pred.flatten()

In [ ]:
# predictions = getSubmission('BayesianRidge')
predictions = getNNSubmission()

output = pd.DataFrame({'Id': test_df.Id, 'SalePrice': np.expm1(predictions)})

output.to_csv('my_submission.csv', index=False)

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler= RobustScaler()
# transform "x_train"
train_input = scaler.fit_transform(train_input)
# transform "x_test"
if (has_cv):
  cv_input = scaler.transform(cv_input)
#Transform the test set
test_data_input = scaler.transform(test_data_input)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

ridge=Ridge()
parameters= {'alpha':[x for x in range(1,101)]}

ridge_reg = GridSearchCV(estimator=ridge, param_grid=parameters, scoring='neg_mean_squared_error', cv=15)
ridge_reg.fit(train_input,train_label)
print("The best value of Alpha is: ",ridge_reg.best_params_)
print("The best score achieved with Alpha=11 is: ",math.sqrt(-ridge_reg.best_score_))
ridge_pred=math.sqrt(-ridge_reg.best_score_)

The best value of Alpha is:  {'alpha': 9}
The best score achieved with Alpha=11 is:  0.11589743355970451


In [ ]:


ridge_mod=Ridge(alpha=11)
ridge_mod.fit(train_input,train_label)
y_pred_train=ridge_mod.predict(train_input)

print('Root Mean Square Error train = ' + str(math.sqrt(mean_squared_error(train_label, y_pred_train))))
if (has_cv):
  y_pred_cv = ridge_mod.predict(cv_input)
  print('Root Mean Square Error test = ' + str(math.sqrt(mean_squared_error(cv_label, y_pred_cv))))

Root Mean Square Error train = 0.10009675674419488


In [ ]:
from sklearn.linear_model import Lasso

parameters= {'alpha':[0.0001,0.0009,0.001,0.002,0.003,0.01,0.1,1,10,100]}

lasso=Lasso()
lasso_reg=GridSearchCV(lasso, param_grid=parameters, scoring='neg_mean_squared_error', cv=15)
lasso_reg.fit(train_input,train_label)

print('The best value of Alpha is: ',lasso_reg.best_params_)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.08305955171536539, tolerance: 0.021675117547308585
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.09122581256003492, tolerance: 0.02174785009373819
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.4532840776144358, tolerance: 0.02182417303907677
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 

The best value of Alpha is:  {'alpha': 0.0009}


In [ ]:
lasso_mod=Lasso(alpha=0.0009)
lasso_mod.fit(train_input,train_label)
y_lasso_train=lasso_mod.predict(train_input)

print('Root Mean Square Error train = ' + str(math.sqrt(mean_squared_error(train_label, y_lasso_train))))
if (has_cv): 
  y_lasso_cv=lasso_mod.predict(cv_input)
  print('Root Mean Square Error test = ' + str(math.sqrt(mean_squared_error(cv_label, y_lasso_cv))))


Root Mean Square Error train = 0.10778109879684361


In [ ]:
# from sklearn.linear_model import ElasticNetCV

# alphas = [0.000542555]
# l1ratio = [0.1, 0.3,0.5, 0.9, 0.95, 0.99, 1]

# elastic_cv = ElasticNetCV(cv=5, max_iter=1e7, alphas=alphas,  l1_ratio=l1ratio)

# elasticmod = elastic_cv.fit(train_input, train_label.ravel())
# if (has_cv):
#   ela_pred=elasticmod.predict(cv_input)
#   print('Root Mean Square Error test = ' + str(math.sqrt(mean_squared_error(cv_label, ela_pred))))
# print(elastic_cv.alpha_)

In [ ]:
from xgboost.sklearn import XGBRegressor

xgb= XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=3, min_child_weight=0, missing=None, n_estimators=4000,
             n_jobs=1, nthread=None, objective='reg:squarederror', random_state=0,
             reg_alpha=0.0001, reg_lambda=0.01, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)
xgmod=xgb.fit(train_input,train_label)
if (has_cv):
  xg_pred=xgmod.predict(cv_input)
  print('Root Mean Square Error test = ' + str(math.sqrt(mean_squared_error(cv_label, xg_pred))))

In [ ]:
from sklearn.ensemble import VotingRegressor


vote_mod = VotingRegressor([('Ridge', ridge_mod), ('Lasso', lasso_mod),
                            ('XGBRegressor', xgb)])
vote= vote_mod.fit(train_input,train_label.ravel())

if (has_cv):
  vote_pred=vote.predict(cv_input)
  print('Root Mean Square Error test = ' + str(math.sqrt(mean_squared_error(cv_label, vote_pred))))

In [ ]:
from mlxtend.regressor import StackingRegressor


stregr = StackingRegressor(regressors=[ridge_mod, lasso_mod, vote_mod], 
                           meta_regressor=xgb, use_features_in_secondary=True
                          )

stack_mod=stregr.fit(train_input,train_label.ravel())
if (has_cv):
  stacking_pred=stack_mod.predict(cv_input)
  print('Root Mean Square Error test = ' + str(math.sqrt(mean_squared_error(cv_label, stacking_pred))))

In [ ]:
if (has_cv):
  vote_testcv = vote_mod.predict(cv_input)
  #StackingRegressor to predict the final Test
  stack_testcv = stregr.predict(cv_input)
  
  #LassoRegressor to predict the final Test
  lasso_testcv = lasso_mod.predict(cv_input)
  
  
  cvcombo = (0.2*vote_testcv+0.6*stack_testcv+0.2*lasso_testcv)
  print(str(math.sqrt(mean_squared_error(cv_label, cvcombo))))


In [ ]:
#VotingRegressor to predict the final Test
vote_test = vote_mod.predict(test_data_input)
final1=np.expm1(vote_test)

#StackingRegressor to predict the final Test
stack_test = stregr.predict(test_data_input)
final2=np.expm1(stack_test)

#LassoRegressor to predict the final Test
lasso_test = lasso_mod.predict(test_data_input)
final3=np.expm1(lasso_test)

In [ ]:
# y_sub = lasso_mod.predict(test_data_input)
y_sub = (0.2*final1+0.6*final2+0.2*final3)

output = pd.DataFrame({'Id': test_df.Id, 'SalePrice': y_sub})

output.to_csv('my_submission.csv', index=False)